In [26]:
import numpy as np
%matplotlib
import matplotlib.pyplot as plt
from hello.pid.lvpid import PIDController
from hello.pid.delay import m2s, s2m, h2s, seconds, minutes, hours, days, DelayBuffer, DelaySink
from matplotlib.ticker import MultipleLocator, FuncFormatter, FormatStrFormatter
from hello.pid.picker import mk_picker

Using matplotlib backend: TkAgg


In [364]:
from decimal import Decimal as D
class DelaySink():
    def __init__(self, delay, initial, pc_decay=0.95, scalar=1):
        assert 0 < pc_decay <= 1, "don't be a retard"
        self.df1 = self.calc_df(delay, pc_decay)
        if delay:
            self.sink = initial * (1-self.df1) / self.df1
        else:
            self.sink = 0
        self._delay = delay
        self._pc = pc_decay
        self.df2 = (scalar - 1)*self.df1
        
    def calc_df(self, delay, pc):
        if not delay:
            return 1
        # Attempt to be precise by temporarily using decimal.Decimal
        one = D(1)
        return float(one - (one - D(pc))**(one / D(delay)))
        
    def set_delay(self, delay):
        self.df = self.calc_df(delay, self._pc)
        self._delay = delay
        
    def delay(self, v):
        """ Delay the value v and advance the decay model by 1 second.
        Add v to the current sink, then allow the sink to decay
        according to the decay factor. 
        
        The total decay is given by sink * (df1 + df2). The value
        returned to the system is given by sink * df1. The amount
        lost (e.g., due to radiative heat losses) is given by 
        sink * df2. 
        
        """
        self.sink += v
        d1 = self.sink*self.df1
        d2 = self.sink*self.df2
        self.sink -= d1 + d2
        return d1
_F = 1

In [340]:
class TempProcess():
    def __init__(self, delay, initial=20, output=0, env=18.5, g=0.0019254, k=-0.001579031, g_mult=1):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial).cycle
        self.g = g / 60
        self.k = k / 60
        self.env = env
        self.pv = initial
        
    def step(self, op):
        op = self.tdelay(op)
        dT = self.pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        self.pv += dpv
        return self.pv
    
class TempProcess2():
    def __init__(self, delay, initial=20, output=0, env=18.5, g=0.0019254, k=-0.001579031, g_mult=1):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.heatsink = DelaySink(delay, output, 0.95, g_mult)
        self.g = g / 60
        self.k = k / 60
        self.env = env
        self.pv = initial
        
    def step(self, op):
        op = self.heatsink.delay(op)
        dT = self.pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        self.pv += dpv
        return self.pv
    
class TempProcess3():
    def __init__(self, delay, initial=20, output=0, env=18.5, g=0.0019254, k=-0.001579031, g_mult=1):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.heatsink = DelaySink(delay, 0, 0.95, g_mult)
        self.g = g / 60
        self.k = k / 60
        self.env = env
        self.pv = initial
        
    def step(self, op):
        dT = self.pv - self.env
        decay = self.k * dT
        gain = self.g * op
        gain = self.heatsink.delay(gain)
        dpv = decay + gain
        self.pv += dpv
        return self.pv

In [231]:
from pysrc.snippets import OptionCategory

offset = 0
p = 60
i = 35
data = []
delay=m2s(15)

class PIDOps(OptionCategory):
    # PID 
    p = 60
    i = 35
    d = 0
    alpha = -1
    linearity = 1
    beta = 1
    gamma = 0
    auto_max = 100
    auto_min = 0
    deadband = 0
    mode = 0
    man_request = 0
    
class TempOps(OptionCategory):    
    
    pid = PIDOps()
    
    # plant    
    k = -0.00357  # C / min * C
    g = 0.002053  # C / min * % 
    k_mult = 1
    g_mult = 1
    delay = m2s(5)
    
    # Simulation
    initial_pv = 20
    env = 20
    initial_output = 0
    setpoint = 37
    end = 5 * hours
    mode = 'o2a'
    process = TempProcess2
    max_iters = 24 * hours
    time_unit = hours
    

In [339]:
from hello.pid.plots import RingBuffer

class DataQueue():
    """ Standard data container for a single set of data.
    Stores permenant data internally in a RingBuffer.
    For performance reasons, store pending values 
    in a simple python list, and push them to the 
    permanent buffer only when explicitly requested. 
    """
    def __init__(self, pts):
        self._values = RingBuffer(pts)
        self._pending = []
        self.put = self._pending.append
        
    def __len__(self):
        return len(self._values)
        
    def push(self):
        self._values.extend(self._pending)
        self._pending.clear()
        self.put = self._pending.append
        
    def get(self):
        self.push()
        return self._values.get()

    def clear(self):
        self._values.clear()
        self._pending.clear()
        
    def resize(self, new_sz):
        self.push()
        nv = RingBuffer(new_sz)
        nv.extend(self.get())
        self._values = nv

        
def _mk_pid(pidops, pv, sp, req, mode):
    c = PIDController(pgain=pidops.p,               itime=pidops.i,
                      dtime=pidops.d,               auto_max=pidops.auto_max,
                      auto_min=pidops.auto_min,       
                      beta=pidops.beta,
                      linearity=pidops.linearity,   alpha=pidops.alpha,
                      deadband=pidops.deadband,     sp_high=100, sp_low=0,
                      gamma=pidops.gamma,           man_request=pidops.man_request,
                      mode=pidops.mode)
    if mode == "o2a":
        c.off_to_auto(pv, sp)
    elif mode == "m2a":
        c.man_to_auto(pv, sp, req)
    elif mode == "a2a":
        c.man_to_auto(pv, pv, req)
    else:
        raise ValueError(mode)
    return c
        
        
class TempSim():
    def __init__(self, ops, pts=None):
        self.ops = ops
        if pts is None:
            pts = ops.end
        self._mi = self.ops.max_iters
        self._data = {}
        self._pts = pts
        vars = [
            "x", "pv", "env", "hd",
            "uk", "up", "ui", "ud",
        ]
        for v in vars:
            self._add_queue(v)
            
        pv = ops.initial_pv
        sp = ops.setpoint
        hd = ops.initial_output
        
        def minmax(v): return min(100, max(v, 0))
        hd = minmax(hd)
            
        self._pid = _mk_pid(ops.pid, pv, sp, hd, ops.mode)
            
        delay = ops.delay
        ProcessClass = ops.process
    
        self._proc = ProcessClass(delay, pv, hd, ops.env, ops.g*ops.g_mult, ops.k*ops.k_mult, ops.g_mult)
        self._time_unit = ops.time_unit
            
        self._state = s = {}
        s['t'] = 0
        s['sp'] = sp
        s['pv'] = pv
        s['k'] = ops.k
        s['g'] = ops.g
        s['hd'] = hd
        s['env'] = ops.env        
        
    @property
    def state(self):
        return self._state.copy()
        
    def _add_queue(self, name):
        if name in self._data:
            raise ValueError(name)
        self._data[name] = DataQueue(self._pts)
        setattr(self, "_"+name, self._data[name])
        
    def getq(self, name):
        return self._data[name]
    
    def get(self, name):
        return self.getq(name).get()
    
    def legacy_data(self):
        """ Sloppily return data in legacy format 
        so I don't have to refactor a bunch of functions. 
        """
        return self._x.get(), self._pv.get(), self._hd.get()
        
    def sim_iters(self, iters=0):
        
        if iters > self._mi:
            iters = self._mi
        if iters <= 0:
            return
            
        # Optimize bytecode a bit...
        _x = self._x
        _pv = self._pv
        _env = self._env
        _hd = self._hd
        _uk = self._uk
        _up = self._up
        _ui = self._ui
        _ud = self._ud

        s = self._state
        t = s['t']
        sp = s['sp']
        env = s['env']
        pv = s['pv']
        
        time_unit = self._time_unit
        proc = self._proc
        pid = self._pid
        
        while iters > 0:
            iters -= 1
            t += 1
            
            op = pid.step(pv, sp)
            pv = proc.step(op)

            _x.put(t/time_unit)
            _pv.put(pv)
            _env.put(proc.env)
            _hd.put(op)

            _uk.put(pid.Uk)
            _up.put(pid.Up)
            _ui.put(pid.Ui)
            _ud.put(pid.Ud)
            
        s['t'] = t
        s['sp'] = sp
        s['pv'] = pv
        s['hd'] = op
        s['env'] = proc.env
        
    def set_time(self, t):
        self._state['t'] = t
        
    def set_pid_value(self, param, value):
        
        if param == "pgain":
            pid.pgain = value
        elif param == "itime":
            pid.itime = m2s(value)
        elif param == "dtime":
            pid.dtime = m2s(value)
        elif param == "beta":
            pid.b = value
        elif param == "mode":
            pid.set_mode(value, pv, sp)
        elif param == "man":
            pid.man_request = float(value)
        else:
            print("Invalid parameter: %r" % param)
    
    def set_value(self, name, value):
        if name in self._state:
            self._state[name] = value
            if name == "env":
                self._proc.env = value
        else:
            print("Error Invalid Value: %r" % name)

    def clear_data(self):
        for q in self._data.values():
            q.clear()
        self._state['t'] = 0

    def resize(self, n):
        for q in self._data.values():
            q.resize(n)

In [90]:
def declare_list(name):
    if name not in globals():
        globals()[name] = []

# Test 1: Initial PID Exploration for 3L

In [91]:
import sys
_g_axes = []
def mk_axes():
    global _g_axes
    _g_axes = []
    i = 1
    while True:
        try:
            ax = globals()['ax' + str(i)]
        except KeyError:
            break
        else:
            _g_axes.append(ax)
        i += 1

def axes():
    return _g_axes

In [376]:
def setup1(new=False):
    global fig, ax1, ax2, ax3, ax4, ax5, ax6
    
    if not plt.get_fignums() or new:
        fig = plt.figure()
        ax1 = fig.add_subplot(2,1,1)
        ax2 = fig.add_subplot(2,1,2)
        mk_axes()
        for a in ax1, ax2:
            b = a.get_position()
            a.set_position([b.x0, b.y0, b.width*0.8, b.height])
            a.grid()
    else:
        for a in axes():
            a.clear()
            a.grid()
        for t in fig.texts:
            t.remove()
    
    global colors, color
    colors = [
        "blue",
        "red",
        "green",
        "cyan",
        "purple",
        "orange",
        "black"
    ]

    import itertools
    color = itertools.cycle(colors).__next__

In [427]:
def temp_sim1(ops):
    sim = TempSim(ops, ops.end)
    sim.sim_iters(ops.end)
    x = sim.get('x')
    pv = sim.get('pv')
    hd = sim.get('hd')
    return x, pv, hd

def _p1(ax, x, y, color, label, ylabel=None):
    ax.plot(x,y, color=color, label=label)
    if ylabel:
        ax.set_ylabel(ylabel)

def plot1(x, pv, hd, label=""):
    c = color()
    _p1(ax1, x, pv, c, label, "Temp (C)")
    _p1(ax2, x, hd, c, label, "Heater Duty (%)")
    
    ax1.axhline(y=ops.setpoint, ls="--", color="black")
    
    ax1.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    ax2.yaxis.set_major_formatter(FormatStrFormatter("%d"))
    
    for a in axes():
        a.xaxis.set_major_locator(MultipleLocator(30))
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
    
    fig.canvas.flush_events()
    fig.canvas.draw()
    
def run1(ops, label=""):
    global x, pv, hd
    x,pv,hd = temp_sim1(ops)
    plot1(x,pv, hd, label)

def finish1():
    for a in axes():
        if a.legend_:
            mk_picker(fig, a)
    

In [388]:
ops = TempOps()

# PID
ops.pid.p = 40
ops.pid.i = 6
ops.pid.d = 0
ops.pid.alpha = -1
ops.pid.linearity = 1
ops.pid.beta = 0
ops.pid.gamma = 0
ops.pid.auto_max = 50
ops.pid.auto_min = 0
ops.pid.deadband = 0
ops.pid.mode = 0
ops.pid.man_request = 0

# plant    
ops.k = -0.004209176  # C / min * C
ops.g = 0.008794422   # C / min * % 
ops.delay = m2s(5)

# Simulation
ops.initial_pv = 22.9
ops.env = 22.9
ops.initial_output = 0
ops.setpoint = 37
ops.end = 2 * hours
ops.mode = 'o2a'
ops.process = TempProcess
ops.max_iters = 24 * hours
ops.time_unit = minutes

In [118]:
setup1()
for d in (3, 5, 10):
    ops.delay = d*minutes
    run1(ops, "D: %d"%d)
finish1()

# Test 2: Comparison of real vs test data

In [284]:
def setup2(new=False):
    setup1(new)
    _p1(ax1, pvx, pvy, "blue", "Real", "Temp (C)")
    _p1(ax2, hdx, hdy, "blue", "Real", "Heat Duty (%)")

def plot2(x, pv, hd, label=""):
    c = "blue"
    while c == "blue":
        c = color()
    _p1(ax1, x, pv, c, label if label else "Sim")
    _p1(ax2, x, hd, c, label if label else "Sim")
    
    ax1.axhline(y=ops.setpoint, ls="--", color="black")
    ax1.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    ax2.yaxis.set_major_formatter(FormatStrFormatter("%d"))
    
    for a in ax1, ax2:
        a.xaxis.set_major_locator(MultipleLocator(30))
        a.legend(bbox_to_anchor=(0.99, 1.06), loc="upper left")
        a.set_xlim(0, 120)
        
def run2(ops, label=""):
    global x, pv, hd
    x,pv,hd = temp_sim1(ops)
    plot2(x, pv, hd, label)
    
finish2 = finish1

# Test 3: PID Tuning with 3L Model

In [410]:
ops = TempOps()

# PID
ops.pid.p = 40
ops.pid.i = 6
ops.pid.d = 0
ops.pid.alpha = -1
ops.pid.linearity = 1
ops.pid.beta = 0
ops.pid.gamma = 0
ops.pid.auto_max = 50
ops.pid.auto_min = 0
ops.pid.deadband = 0
ops.pid.mode = 0
ops.pid.man_request = 0

# plant    
ops.k = -0.004209176  # C / min * C
ops.g = 0.008794422   # C / min * % 
ops.delay = m2s(15)

# Simulation
ops.initial_pv = 22.9
ops.env = 22.9
ops.initial_output = 0
ops.setpoint = 37
ops.end = 2 * hours
ops.mode = 'o2a'
ops.process = TempProcess2
ops.max_iters = 24 * hours
ops.time_unit = minutes
ops.g_mult = 1.2
ops.k_mult = 0.5

In [431]:
setup3=setup1
plot3=plot1
def run3(ops, lbl):
    global x,pv,hd
    x,pv,hd = temp_sim1(ops)
    plot3(x,pv,hd,lbl)
def plot3(*args):
    plot1(*args)
    ax1.set_ylim(36, 38)
finish3=finish1    

In [442]:
setup3()
for i in (6, 8, 9):
    ops.pid.p = 40
    ops.pid.i = i
    run3(ops, "i:%d"%i)
finish3()

In [449]:
setup3()
for p in (10, 20, 14):
    ops.pid.p = p
    ops.pid.i = 20
    run3(ops, "P:%d"%p)
finish3()

In [450]:
setup3()
ops.pid.p = 14
ops.pid.i = 20
for env in 16, 20, 24:
    ops.env = env
    run3(ops, "E:%d"%env)
finish3()
ops.env=22.9

# Testing with Excel data for side-by-side comparison

In [108]:
from officelib.xllib import *
xl = Excel()
wb = xl.ActiveWorkbook
cells = wb.ActiveSheet.Cells
cell_range = cells.Range

In [192]:
def paste():
    x1 = map(float, x)
    pv1 = map(float, pv)
    hd1 = map(float, hd)
    data = list(zip(x1, pv1, hd1))
    c1 = cell_range("EG2")
    c2 = c1.Offset(len(data), len(data[0]))
    cell_range(c1, c2).Value = data

In [402]:
base_g = .397637759 * 1
amax = 50
base_k = -0.004209176

env = 22.9
dt = (37+22)/2 - env
ops.g = (base_g - dt * base_k) / amax
ops.k = base_k
ops.delay = 18 * minutes
ops.process = TempProcess2

setup1()
_F = 0
run1(ops, "test")
_F = .5
run1(ops, "test2")
paste()

In [399]:
def get(r):
    return [np.array(d) for d in list(zip(*cell_range(r).Value))]
pvx, pvy = get("DY2:DZ998")
hdx, hdy = get("DO2:DP930")

In [390]:
base_g = .397637759 * 1
amax = 50
base_k = -0.004209176

env = 22.9
dt = (37+22)/2 - env
ops.g = (base_g - dt * base_k) / amax
ops.k = base_k
ops.delay = 18 * minutes
ops.process = TempProcess2

setup2()
ops.g_mult = 1.4
_F = 1/2.5
run2(ops)
paste()

In [398]:
setup2()
ops.env=22.9
ops.delay = 15*minutes
ops.process = TempProcess2
for m in 0.2, 0.4, 0.6:
    ops.g_mult = 1 + m
    _F = m
    run2(ops, "M:%.2f"%(m))
finish2()

In [384]:
setup2()
d = 20
ops.delay = 18*minutes
ops.g_mult = 1.5
_F = 0.5
ops.process = TempProcess2
run2(ops, "P1")
ops.process = TempProcess3
run2(ops, "P2")
finish2()

In [409]:
setup2()
ops.env = 22.9
ops.delay=15*minutes
ops.process = TempProcess2
ops.k_mult = 0.5
ops.g_mult = 1.2
run2(ops, "test")
finish2()

In [404]:
ops

pid: <More Options...>
k: -0.004209176
g: 0.008508366412
k_mult: 1
g_mult: 1.2
delay: 900
initial_pv: 22.9
env: 22.9
initial_output: 0
setpoint: 37
end: 7200
mode: o2a
process: <class '__main__.TempProcess2'>
max_iters: 86400
time_unit: 60